In [7]:
import numpy as np
from PIL import Image # Thêm thư viện Pillow để đọc ảnh

# --- Class GMM Đã Tối Ưu Hóa (Vector hóa) ---
class GMM:
    def __init__(self, n_components, max_iter=100, comp_names=None, tol=1e-4):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol # Ngưỡng hội tụ

        if comp_names is None:
            self.comp_names = [f"comp_{i}" for i in range(n_components)]
        else:
            self.comp_names = comp_names

        self.pi = None # Sẽ khởi tạo trong fit
        self.means = None
        self.covs = None
        self.gamma = None # responsibilities

    def multivariate_normal_pdf(self, X, mean_vector, covariance_matrix):
        """
        Tính PDF của phân phối Gauss đa biến.
        X: (N_samples, D_features) hoặc (D_features,)
        mean_vector: (D_features,)
        covariance_matrix: (D_features, D_features)
        """
        d = mean_vector.shape[0]

        # Đảm bảo ma trận hiệp phương sai không suy biến
        epsilon_stabilizer = 1e-6 
        stable_cov = covariance_matrix + np.eye(d) * epsilon_stabilizer

        try:
            inv_cov = np.linalg.inv(stable_cov)
            det = np.linalg.det(stable_cov)
            if det <= 0:
                det = 1e-10 # Gán giá trị rất nhỏ nếu định thức không dương
        except np.linalg.LinAlgError:
            # Trả về 0 hoặc một giá trị rất nhỏ nếu ma trận vẫn suy biến
            return np.zeros(X.shape[0]) if X.ndim > 1 else 1e-300

        norm_const = 1.0 / (np.power((2 * np.pi), d / 2) * np.sqrt(det))

        if X.ndim == 1:
            # Xử lý khi X là một điểm dữ liệu đơn lẻ (vector 1D)
            diff = X - mean_vector
            exponent = -0.5 * np.dot(np.dot(diff.T, inv_cov), diff)
            if exponent < -700: # Tránh tràn số âm
                return 1e-300
            return norm_const * np.exp(exponent)
        else:
            # Xử lý khi X là một mảng các điểm dữ liệu (vector hóa)
            diff = X - mean_vector # (N_samples, D_features)
            # Tính (x-mu)T * inv_cov * (x-mu) cho mỗi hàng
            exponent = -0.5 * np.sum(np.dot(diff, inv_cov) * diff, axis=1) # (N_samples,)
            
            # Tránh tràn số âm cho toàn bộ mảng
            exponent[exponent < -700] = -700 # Clip giá trị exponent để tránh exp(rất_âm) -> 0
            
            return norm_const * np.exp(exponent)

    def fit(self, X):
        n_samples, n_features = X.shape

        np.random.seed(42)
        # Khởi tạo means: Chọn ngẫu nhiên n_components điểm từ X
        self.means = [X[np.random.choice(n_samples)] for _ in range(self.n_components)]
        # Khởi tạo covs: Mỗi cụm bắt đầu với một ma trận đơn vị nhỏ
        self.covs = [np.eye(n_features) * 0.1 for _ in range(self.n_components)]
        # Khởi tạo pi (weights) đồng đều
        self.pi = np.ones(self.n_components) / self.n_components

        self.gamma = np.zeros((n_samples, self.n_components)) # responsibilities
        
        prev_log_likelihood = -np.inf # Theo dõi log-likelihood để kiểm tra hội tụ

        for iteration in range(self.max_iter):
            # E-step: tính xác suất posterior (responsibilities) - Đã vector hóa
            component_pdfs = np.zeros((n_samples, self.n_components))
            for k in range(self.n_components):
                # Gọi hàm multivariate_normal_pdf đã được vector hóa
                component_pdfs[:, k] = self.multivariate_normal_pdf(X, self.means[k], self.covs[k])
            
            numerator = component_pdfs * self.pi # broadcasting pi (1, K) với (N, K)

            sum_probs = np.sum(numerator, axis=1, keepdims=True)
            # Tránh chia 0 nếu sum_probs rất nhỏ
            sum_probs[sum_probs == 0] = 1e-10

            self.gamma = numerator / sum_probs # responsibilities = gamma(z_ik)

            # Tính log-likelihood cho lần lặp hiện tại
            current_log_likelihood = np.sum(np.log(sum_probs))

            # M-step: cập nhật tham số
            N_k = np.sum(self.gamma, axis=0) # Tổng responsibilities cho mỗi cụm (N_k)

            # Cập nhật pi
            self.pi = N_k / n_samples
            # Đảm bảo pi không quá nhỏ và tổng bằng 1
            self.pi[self.pi < 1e-10] = 1e-10 
            self.pi /= np.sum(self.pi) 

            for k in range(self.n_components):
                # Tránh chia cho 0 nếu một cụm không có điểm nào được gán hiệu quả
                if N_k[k] < 1e-10: 
                    continue 
                
                # Update mean
                self.means[k] = np.sum(self.gamma[:, k][:, np.newaxis] * X, axis=0) / N_k[k]

                # Update covariance
                diff = X - self.means[k] 
                self.covs[k] = np.dot((self.gamma[:, k][:, np.newaxis] * diff).T, diff) / N_k[k]
                
                # Đảm bảo tính ổn định số học cho covs sau cập nhật
                self.covs[k] += np.eye(n_features) * 1e-6 
            
            # Kiểm tra hội tụ bằng log-likelihood
            if abs(current_log_likelihood - prev_log_likelihood) < self.tol:
                print(f"GMM converged at iteration {iteration + 1}")
                break
            prev_log_likelihood = current_log_likelihood

    def predict(self, X):
        n_samples = X.shape[0]
        
        # Tính xác suất mỗi điểm thuộc về từng thành phần - Đã vector hóa
        component_probs = np.zeros((n_samples, self.n_components))
        for k in range(self.n_components):
            component_probs[:, k] = self.pi[k] * self.multivariate_normal_pdf(X, self.means[k], self.covs[k])

        # Tránh lỗi nếu tất cả probs đều 0 cho một điểm
        sum_component_probs = np.sum(component_probs, axis=1, keepdims=True)
        sum_component_probs[sum_component_probs == 0] = 1e-10
        
        # Xác suất posterior (gamma_hat)
        gamma_hat = component_probs / sum_component_probs

        # Nhãn dự đoán là thành phần có xác suất cao nhất
        y_pred = np.argmax(gamma_hat, axis=1)

        return np.array(y_pred)

In [15]:
# 1. Đọc ảnh của bạn và chuyển đổi sang mảng NumPy
# BẠN CẦN THAY ĐỔI ĐƯỜNG DẪN DƯỚI ĐÂY THÀNH ĐƯỜNG DẪN FILE ẢNH 'cow.jpg' CỦA BẠN
image_path = 'D:/LAP TRINH/DS102.P21.2/LAB_5/cow.jpg' 

try:
    img = Image.open(image_path)
    if img.mode != 'RGB': # Đảm bảo ảnh là RGB
        img = img.convert('RGB')
    image_np = np.array(img)
    
    height, width, channels = image_np.shape
    print(f"Đã đọc ảnh thành công. Kích thước ảnh: {image_np.shape}\n")

except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file ảnh tại đường dẫn '{image_path}'")
    print("Vui lòng kiểm tra lại đường dẫn file ảnh của bạn.")
    exit() 
except Exception as e:
    print(f"Lỗi khi đọc ảnh: {e}")
    exit() 

# 2. Tiền xử lý dữ liệu ảnh cho GMM
pixels = image_np.reshape(-1, channels).astype(np.float64)
pixels_normalized = pixels / 255.0

print(f"Dữ liệu pixel sau khi reshape và chuẩn hóa: {pixels_normalized.shape}\n")

# 3. Huấn luyện mô hình GMM trên dữ liệu pixel
# Thử nghiệm với các giá trị khác nhau cho K (ví dụ: 3, 4, 5, 6)
n_components_image_gmm = 3 

print(f"Bắt đầu huấn luyện GMM cho ảnh với K = {n_components_image_gmm}...")
# max_iter có thể giữ 200, vì có tol sẽ dừng sớm nếu hội tụ
gmm_image = GMM(n_components=n_components_image_gmm, max_iter=200, tol=1e-4) 
gmm_image.fit(pixels_normalized)
print("Huấn luyện GMM cho ảnh hoàn tất.\n")

# Dự đoán nhãn cụm cho mỗi pixel
pixel_labels = gmm_image.predict(pixels_normalized)

# Reshape các nhãn trở lại kích thước ảnh ban đầu
segmentation_map = pixel_labels.reshape(height, width)

print("Bản đồ phân đoạn (nhãn cụm của từng pixel, 5x5 đầu tiên):\n")
print(segmentation_map[0:5, 0:5]) 
print("...\n")

# 4. Xác định thành phần nền và lọc
# Heuristic: Các pixel ở 4 góc ảnh thường thuộc về nền.
corner_pixels_data = np.array([
    pixels_normalized[0],                       
    pixels_normalized[width - 1],               
    pixels_normalized[(height - 1) * width],    
    pixels_normalized[height * width - 1]       
])

corner_labels_predicted = gmm_image.predict(corner_pixels_data)

unique_labels, counts = np.unique(corner_labels_predicted, return_counts=True)
if len(unique_labels) > 0:
    most_common_corner_label = unique_labels[np.argmax(counts)]
else:
    most_common_corner_label = 0 

background_label = most_common_corner_label
print(f"Nhãn cụm nền được xác định bằng heuristic (pixel góc): {background_label}")
print(f"Giá trị trung bình màu của cụm nền này (chuẩn hóa 0-1): {gmm_image.means[background_label]}")
print(f"Giá trị trung bình màu của cụm nền này (nguyên bản 0-255): {gmm_image.means[background_label] * 255}\n")


# 5. Tạo ảnh đã lọc nền
filtered_image_np = np.copy(image_np) 

foreground_mask = (pixel_labels != background_label).reshape(height, width)

filtered_image_np[~foreground_mask] = [0, 0, 0] 

print(f"Kích thước ảnh đã lọc nền: {filtered_image_np.shape}\n")

# Lưu ảnh đã lọc nền
try:
    img_filtered = Image.fromarray(filtered_image_np)
    output_path = 'D:/LAP TRINH/DS102.P21.2/LAB_5/cow_filtered_background_optimized.png' # Đường dẫn lưu ảnh
    img_filtered.save(output_path)
    print(f"Ảnh đã lọc nền được lưu tại: {output_path}")
    print("Vui lòng mở file này để xem kết quả.")
except Exception as e:
    print(f"Lỗi khi lưu ảnh đã lọc nền: {e}")

Đã đọc ảnh thành công. Kích thước ảnh: (196, 300, 3)

Dữ liệu pixel sau khi reshape và chuẩn hóa: (58800, 3)

Bắt đầu huấn luyện GMM cho ảnh với K = 3...
GMM converged at iteration 69
Huấn luyện GMM cho ảnh hoàn tất.

Bản đồ phân đoạn (nhãn cụm của từng pixel, 5x5 đầu tiên):

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
...

Nhãn cụm nền được xác định bằng heuristic (pixel góc): 0
Giá trị trung bình màu của cụm nền này (chuẩn hóa 0-1): [0.26982143 0.34485918 0.18486579]
Giá trị trung bình màu của cụm nền này (nguyên bản 0-255): [68.80446469 87.93909093 47.1407771 ]

Kích thước ảnh đã lọc nền: (196, 300, 3)

Ảnh đã lọc nền được lưu tại: D:/LAP TRINH/DS102.P21.2/LAB_5/cow_filtered_background_optimized.png
Vui lòng mở file này để xem kết quả.
